# Load training and test data

First, we want to load the training and test data and preprocess it before use. The preprocessing is done using `torchvision.transforms.Compose`, which allows a series of transformations to be applied. The main preprocessing is simply normalization, using `torchvision.transforms.Normalize`. The `ToTensor` converts the numpy arrays to tensors.

In [1]:
import torch
import torchvision

# set up some transforms
torch_transforms = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.1307,), (0.3081,))]
)

# Download mnist data, preprocess, save to directory `./files`
mnist_train = torchvision.datasets.MNIST('./files/', train=True, download=True, transform=torch_transforms)
mnist_test = torchvision.datasets.MNIST('./files/', train=False, download=True, transform=torch_transforms)

# create the data loader objects; this creates an iterable
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=20, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size = 1000, shuffle=True)

# Create conv net

In [2]:
from torch import nn
from torch.nn import functional as F
from torch.autograd import Variable
import torch.optim as optim

class ConvNet(nn.Module):
    
    def __init__(self):
        super(ConvNet, self).__init__()
        
        # 1 input channel, 10 output channels, convolving kernel of size 5
        self.conv_layer1 = nn.Conv2d(1, 10, kernel_size=5)
        
        # 10 input channel, 20 output channels, convolving kernel of size 5
        self.conv_layer2 = nn.Conv2d(10, 20, kernel_size=5)
        
        # Randomly zero out channels, for regularization
        self.conv_layer2_dropout = nn.Dropout2d()
        
        # linear layer: 320 units to 50
        self.linear_layer_1 = nn.Linear(320, 50)
        
        # final layer: 50 units to 10;
        # 10 is number of classes
        # softmax will be applied
        self.linear_layer_2 = nn.Linear(50, 10)

    def forward(self, x):
        # forward pass
        # using rectified linear unit activation function
        # applied to max pooling
        x = F.relu(
                F.max_pool2d(
                    self.conv_layer1(x), 2))
        
        x = F.relu(
                F.max_pool2d(
                    self.conv_layer2_dropout(
                        self.conv_layer2(x)
                    ), 2)
            )
        
        x = x.view(-1, 320) # reshape to 320 columns
        x = F.relu(self.linear_layer_1(x))
        x = F.dropout(x, 
                      training=self.training)
        x = self.linear_layer_2(x)
        
        return F.log_softmax(x, dim = 1)

In [3]:
convnet = ConvNet()

optimizer = optim.SGD(convnet.parameters(), lr = 0.01)

In [4]:
# Iterate
for iteration in range(10):
    for batch_idx, (data, target) in enumerate(train_loader):

        optimizer.zero_grad()
        output = convnet(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 500 == 0:
            print(loss.item())

2.263284206390381
1.0636253356933594
0.4785844385623932
0.8917247653007507
0.2977626919746399
0.3288015127182007
0.7607751488685608
0.5436597466468811
0.15207163989543915
0.4659230709075928
0.5002973079681396
0.09498219192028046
0.3240170478820801
0.4145418107509613
0.24259385466575623
0.14451900124549866
0.20439867675304413
0.16882291436195374
0.2743907570838928
0.2627004384994507
0.181642085313797
0.051711685955524445
0.40202587842941284
0.47962814569473267
0.2670667767524719
0.21650806069374084
0.12450365722179413
0.10079549252986908
0.13512983918190002
0.10721735656261444
0.08505983650684357
0.19369365274906158
0.12274066358804703
0.032289229333400726
0.12263616174459457
0.5560099482536316
0.010165827348828316
0.42380914092063904
0.19152328372001648
0.08325190097093582
0.0907195508480072
0.24130377173423767
0.05588749051094055
0.18094287812709808
0.3862128257751465
0.09364720433950424
0.34869492053985596
0.14530973136425018
0.015458449721336365
0.06244869902729988
0.247671008110046